# Generate CMA using daily data service - Python

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *cumulative moving average - CMA* from a `daily data service`daily data service.

### Inputs/outputs
Cumulative moving averages' requests require instrument's identifier, date time intervals and number of periods as per inputs.

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve daily data from the hosted service. The queried endpoint in this script are:

### Modules required
1. Systemathics:
     * *systemathics.apis.services.daily_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *matplotlib* as per display package

***

# Run daily CMA sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly matplotlib systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.services.daily_analytics.v1.daily_cma_pb2 as daily_cma
import systemathics.apis.services.daily_analytics.v1.daily_cma_pb2_grpc as daily_cma_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Retrieve data
To request *daily cma* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'XNGS'

#### 3.2 Time period selection

In [ ]:
# set data interval (we are using Google date time format) 
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2019, month = 1, day = 1),
    end_date = date.Date(year = 2020, month = 12, day = 31)
)

# generate constraints based on the previous date selection
# time constraints are not neeeded for the dividends
constraints = constraints.Constraints(
    date_intervals = [date_interval]
)

#### 3.3 CMA request creation
The following code snippet creates *gRPC client*, process daily CMA request and ensure that the reply is not empty:

In [ ]:
# generate daily CMA request
request = daily_cma.DailyCmaRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker), 
    constraints = constraints
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:
    
    # instantiate the daily cma service
    service = daily_cma_service.DailyCmaServiceStub(channel)
    
    # process the daily CMA request
    response = service.DailyCma(
        request = request,
        metadata = [('authorization', token)]
    )

The following code snippet displays total data points count:

In [ ]:
print("Total data points retrieved for CMA: ", len(response.data))

### Step 4: Visualize data

#### 4.1 Retrieve CMA and daily prices data
In the following code snippets, the daily CMA reply is reprocessed in a dataframe in order to visualize the results:

In [ ]:
# create the  dataframe
dates = [datetime(l.date.year, l.date.month, l.date.day) for l in response.data]
prices = [l.value for l in response.data]
cmas = [l.average for l in response.data]

d = {'Date': dates, 'Price': prices, 'CMA': cmas}
df = pd.DataFrame(data=d)
df

#### 4.2 Plot CMA and daily prices with matplotlib

In [ ]:
import matplotlib.pyplot as plt

# create figure and axis objects with subplots()
fig,ax = plt.subplots(1,1,figsize=(25,10))
ax.plot('Date', 'CMA', data=df, marker='', color='orange', linewidth=2, label="CMA")
ax.plot('Date', 'Price', data=df, marker='', color='blue', linewidth=2, label="Price")

ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="black",fontsize=14)
plt.legend()
plt.title('CMA and daily prices bars for {0}-{1}'.format(ticker,exchange))
plt.show()

#### 4.3 Plot CMA and daily prices with plotly

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Date'], y=df['CMA'], mode='lines', name='CMA'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'], mode='lines', name='Price'))

fig.update_layout(title = 'CMA and daily prices bars for {0}-{1}'.format(ticker,exchange))
fig.update_xaxes(title_text = 'Date')
fig.update_yaxes(title_text = 'Price')
fig.show()